In [1]:
import sys
import json
from pathlib import Path
from google.oauth2.credentials import Credentials

root_dir = Path("/workspaces/DeliveryTracker")

sys.path.insert(0, str(root_dir))


credential_json = json.load((root_dir / "google_token/logged_in_credentials.json").open())
credentials = Credentials.from_authorized_user_info(credential_json)

In [2]:
from shared.GoogleServices import GmailService

In [3]:
gs = GmailService(credentials)
gs.authenticate()

In [4]:
from datetime import datetime, timedelta, timezone


time_threshold = datetime.now(timezone.utc) - timedelta(days=100)
query = f"from:no-reply@winsim.de subject:'Ihre winSIM-Rechnung' after:{int(time_threshold.timestamp())}"
    
    # 2. Use the Gmail API to find message IDs that match the query.
response = gs.service.users().messages().list(userId='me', q=query).execute()
messages = response.get('messages', [])

In [5]:
messages

[{'id': '1945b4defe14256d', 'threadId': '1945b4defe14256d'},
 {'id': '193bb43a046d7385', 'threadId': '193bb43a046d7385'},
 {'id': '1932448bbe10738f', 'threadId': '1932448bbe10738f'}]

In [6]:
gs.download_pdf_attachments(messages[0]['id'])

['INVB790273815.pdf']

In [7]:
msg = messages[0]

msg_detail = gs.service.users().messages().get(userId='me', id=msg['id'], format='full').execute()

In [ ]:
parts = msg_detail['payload'].get('parts', [])
parts

In [9]:
for part in parts:
    filename: str = part['filename']
    if filename.endswith(".pdf"):
        print(filename)
        break

INVB790273815.pdf


In [10]:
attach_id = part['body']['attachmentId']

attachment = gs.service.users().messages().attachments().get(
            userId='me',
            messageId=msg['id'],
            id=attach_id
        ).execute()

In [11]:
import base64
import os


file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
filepath = "testi_attachment.pdf"
with open(filepath, 'wb') as f:
    f.write(file_data)